In [9]:
import numpy as np
import keras as K
import tensorflow as tf
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [10]:
np.random.seed(1)
print(tf.__version__)

df = pd.read_csv("Gungor_2018_VictorianAuthorAttribution_data-train.csv")

df['author'].unique()

1.14.0


array([ 1,  2,  3,  4,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 50])

In [11]:
p = np.random.permutation(df['author'].unique())
trainclass = p[:40]
fewclass = p[40:]

trainingclasses = df[df['author'].isin(trainclass)]

fewclasses = df[df['author'].isin(fewclass)]


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential, Model

from keras.layers import *
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional


os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [13]:
x = np.array(trainingclasses['text'])
y = np.array(pd.factorize(trainingclasses['author'])[0])

In [167]:
max_words = 20000
                
tokenizer = Tokenizer(20000)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

train_x, test_x, train_y, test_y = train_test_split(sequences, y, test_size=0.33, random_state=42)
max_review_len = 1000
train_x = K.preprocessing.sequence.pad_sequences(train_x,
  truncating='pre', padding='pre', maxlen=max_review_len)  # pad and chop!
test_x = K.preprocessing.sequence.pad_sequences(test_x,
  truncating='pre', padding='pre', maxlen=max_review_len)

In [168]:
sequences[0]

[2310,
 28,
 75,
 4,
 1117,
 6,
 59,
 195,
 13,
 1,
 1962,
 449,
 8,
 35,
 10,
 112,
 515,
 1,
 4356,
 3,
 5,
 827,
 2644,
 2,
 1460,
 174,
 15,
 1085,
 15,
 44,
 3,
 67,
 106,
 4835,
 6,
 19,
 1,
 75,
 4,
 1117,
 3,
 238,
 44,
 77,
 75,
 18,
 261,
 2,
 438,
 1185,
 7,
 1,
 250,
 171,
 4,
 292,
 1,
 1204,
 11,
 7,
 5,
 7514,
 2,
 17,
 31,
 84,
 27,
 1,
 909,
 17,
 1,
 4545,
 47,
 277,
 58,
 26,
 177,
 27,
 5,
 1230,
 4,
 1,
 15,
 1,
 9388,
 11,
 82,
 205,
 144,
 52,
 47,
 636,
 1,
 145,
 47,
 9279,
 1,
 4388,
 152,
 47,
 708,
 45,
 18,
 40,
 1077,
 3807,
 27,
 1,
 113,
 128,
 4,
 31,
 254,
 573,
 2,
 1214,
 1,
 449,
 10,
 11,
 244,
 174,
 3492,
 18,
 421,
 2,
 6,
 226,
 31,
 254,
 17,
 2011,
 9,
 6,
 90,
 26,
 177,
 191,
 10,
 7,
 1668,
 18,
 31,
 2910,
 6,
 35,
 6,
 90,
 133,
 5,
 214,
 9790,
 7,
 10,
 18,
 1,
 780,
 3,
 3639,
 448,
 24,
 7,
 1,
 1030,
 50,
 863,
 1,
 1088,
 8,
 19,
 347,
 37,
 10,
 50,
 5358,
 31,
 2437,
 41,
 6,
 4,
 453,
 27,
 34,
 1968,
 1,
 411,
 9,
 96,
 25,
 88

In [150]:
x.shape

(48472,)

In [151]:
x2 = np.array(fewclasses['text'])
y2 = np.array(pd.factorize(fewclasses['author'])[0])

In [152]:
x2.shape

(5206,)

In [153]:
from keras.models import load_model

model = load_model('model_fewshot_1.h5')

/home/tokyotechies/miniconda3/envs/author_classification/lib/python3.6/site-packages/keras/engine/training_utils.py:819: UserWarning: Output features missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to features.
  'be expecting any data to be passed to {0}.'.format(name))


In [154]:
x3 = []

for i in range(5):
    x3.append(np.random.choice(x2[y2==i], size=5))

In [155]:
print(train_x.shape)
print(x2.shape)
print(np.array(x3).ravel().shape)

(32476, 1000)
(5206,)
(25,)


In [156]:
x3 = np.array(x3).ravel()

In [157]:
sequences = tokenizer.texts_to_sequences(x3)
test_x3 = tokenizer.texts_to_sequences(x2)



max_review_len = 1000
train_x = K.preprocessing.sequence.pad_sequences(sequences,
  truncating='pre', padding='pre', maxlen=max_review_len)  # pad and chop!
test_x = K.preprocessing.sequence.pad_sequences(test_x3,
  truncating='pre', padding='pre', maxlen=max_review_len)

In [158]:
train_features, _ = model.predict(train_x)

In [159]:
test_features, _ = model.predict(np.array(test_x3))

In [160]:
train_features = np.expand_dims(train_features, axis=0)
test_features = np.expand_dims(test_features, axis=1)

In [161]:


print(train_features.shape)
print(test_features.shape)

(1, 25, 40)
(5206, 1, 40)


In [162]:
dif = train_features - test_features

In [163]:
print(dif.shape)

(5206, 25, 40)


In [164]:


distance = np.linalg.norm(dif, axis=2)
#predictions = np.argmin(distance, axis=1)
index = np.argpartition(distance, 5, axis=1)
values = index[:,:5]//5
predictions = []
for row in values:
    unique, counts = np.unique(row, return_counts=True)
    prediction = unique[np.argmax(counts)]
    predictions.append(prediction)

predictions = np.array(predictions)


print(predictions.shape)


(5206,)


In [165]:
np.sum(predictions == y2)/5206

0.2656550134460238

In [110]:
predictions[:10]

array([2, 2, 4, 3, 3, 4, 1, 4, 4, 0])

In [100]:
unique.shape

(1325, 5)

In [84]:
min(predictions)

0

0.33480599308490205

In [50]:
model.save('model_fewshot_1.h5')

In [151]:
min(y2)

0